## Imports

In [2]:
import pandas as pd

## Gather Data

In [ ]:
# Setting base URL for the Pushift API to work with
# url = 'https://api.pushshift.io/reddit/search/submission?'

In [ ]:
# Initial parameters for requests to implement
# params = {
#         'subreddit': 'liberal',
#         'size': 100
# }

In [ ]:
# Test gathering data from one pull from Pushift API
# posts = requests.get(url, params).json()
# df = pd.DataFrame(posts['data'])

In [ ]:
# df['created_utc'].min()

In [ ]:
# Creating a function to pull 2500 submissions from our subreddit
# def reddit_api(subreddit):
#     # User only needs subreddit name as a string for input
#     params = {
#         'subreddit': subreddit,
#         'size': 100
#     }
#     # Empty list to store scraped dataframes before combining
#     df = []
#     # Loop 25 times to pull 100 posts each loop: total 2500 posts
#     while len(df) < 25:
#         posts = requests.get(url, params).json()
#         to_concat = pd.DataFrame(posts['data'])
#         params['before'] = to_concat['created_utc'].min()
#         df.append(to_concat)
        
#     return pd.concat(df, axis=0)

In [ ]:
# Commented out because I keep accidentally running
#lib = reddit_api('liberal')
#con = reddit_api('conservative')

In [ ]:
# Save imports to access more easily later.
# lib.to_csv('./data/r_liberal.csv', index=False)
# con.to_csv('./data/r_conservative.csv', index=False)

In [3]:
lib = pd.read_csv('./data/r_liberal.csv')
con = pd.read_csv('./data/r_conservative.csv')

In [4]:
# Checking that our pull was successful with no significant duplicates
print('r/liberal duplicated')
print()
print(lib['created_utc'].duplicated().value_counts())
print('-----------------------------------')
print('r/conservative duplicated')
print()
print(con['created_utc'].duplicated().value_counts())

r/liberal duplicated

False    2500
Name: created_utc, dtype: int64
-----------------------------------
r/conservative duplicated

False    2460
True       40
Name: created_utc, dtype: int64


## Begin EDA and data Cleaning

In [5]:
pd.options.display.max_columns=80

In [5]:
def col_len(x,y):
    if len(x.columns) < len(y.columns):
        drop_columns = list(set(y) - set(x))
    elif len(x.columns) > len(y.columns):
        drop_columns = list(set(x) - set(y))

    return drop_columns


In [6]:
# drops = col_len(lib, con)

In [7]:
# con = con.drop(columns=drops)
# len(con.columns)

76

In [8]:
# Comparing null values between both subs
# list(zip(lib.columns, lib.isnull().sum(), con.isnull().sum()))

[('all_awardings', 0, 0),
 ('allow_live_comments', 0, 0),
 ('author', 0, 0),
 ('author_flair_css_class', 2500, 2500),
 ('author_flair_richtext', 106, 2409),
 ('author_flair_text', 2499, 245),
 ('author_flair_type', 106, 1441),
 ('author_fullname', 106, 1196),
 ('author_patreon_flair', 106, 245),
 ('author_premium', 106, 245),
 ('awarders', 0, 245),
 ('can_mod_post', 0, 245),
 ('contest_mode', 0, 0),
 ('created_utc', 0, 0),
 ('domain', 0, 0),
 ('full_link', 0, 0),
 ('gildings', 0, 0),
 ('id', 0, 0),
 ('is_crosspostable', 0, 0),
 ('is_meta', 0, 0),
 ('is_original_content', 0, 0),
 ('is_reddit_media_domain', 0, 0),
 ('is_robot_indexable', 0, 0),
 ('is_self', 0, 0),
 ('is_video', 0, 0),
 ('link_flair_background_color', 2500, 0),
 ('link_flair_richtext', 0, 0),
 ('link_flair_text_color', 0, 2500),
 ('link_flair_type', 0, 0),
 ('locked', 0, 0),
 ('media_only', 0, 0),
 ('no_follow', 0, 0),
 ('num_comments', 0, 2407),
 ('num_crossposts', 0, 0),
 ('over_18', 0, 0),
 ('parent_whitelist_status', 

In [9]:
def df_concat(x, y):
    keep_columns = ['subreddit', 'title',  'author', 'domain', 'id', 'num_comments', 
                'score', 'selftext', 'url']
    df = pd.concat([lib, con], ignore_index=True)
    df = df[keep_columns]
    df['subreddit'] = df['subreddit'].apply(lambda x: 1 if x == 'Liberal' else 0)
    df['title_word_count'] = [len(title.split()) for title in df['title']]
    return df

In [10]:
# df = df_concat(lib, con)
# df.head()

,subreddit,title,author,domain,id,num_comments,score,selftext,url,title_word_count
0,1,"On behalf of my neanderthal ancestors, I object!",MisanthropicScott,self.Liberal,ly2nha,0,1,[removed],https://www.reddit.com/r/Liberal/comments/ly2n...,8
1,1,You Can't Change This Hatred—You Have to Outnu...,moochir,johnpavlovitz.com,lxy58f,19,181,NaN,https://johnpavlovitz.com/2018/08/07/you-cant-...,9
2,1,Black teen fails all but 3 classes in 4 years ...,XiLin02,foxbaltimore.com,lxx3np,0,1,NaN,https://foxbaltimore.com/news/project-baltimor...,23
3,1,How The Press Enables The GOP’s Big Lie,NORDLAN,nationalmemo.com,lxvws3,0,1,NaN,https://www.nationalmemo.com/election-2020-lies-,8
4,1,Russian State TV Is Really Missing Trump Now B...,NORDLAN,thedailybeast.com,lxvnt0,2,5,NaN,https://www.thedailybeast.com/russian-state-tv...,11


In [81]:
df.to_csv('./data/subreddit_complete.csv')